In [37]:
## point-wise(직접 구현) , depth-wise(내장된 거 사용)

In [1]:
import tensorflow as tf
import numpy as np

2025-07-14 10:00:36.678892: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-14 10:00:36.789675: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-14 10:00:36.818346: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (3.0.4) doesn't match a supporte

In [2]:
mnist=tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test)=mnist.load_data()

In [3]:
print(x_train.shape)

(60000, 28, 28)


In [4]:
x_train=tf.expand_dims(x_train,axis=3)

2025-07-14 10:00:38.446802: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-14 10:00:38.899264: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22294 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:17:00.0, compute capability: 8.6


In [5]:
x_train.shape

TensorShape([60000, 28, 28, 1])

In [6]:
x_train_resize=tf.image.resize(x_train,(224,224))

In [7]:
x_train_resize.shape

TensorShape([60000, 224, 224, 1])

In [8]:
len(np.unique(y_test))

10

In [9]:
x_test=tf.expand_dims(x_test,axis=-1)

In [10]:
x_test.shape

TensorShape([10000, 28, 28, 1])

In [11]:
x_test_resize=tf.image.resize(x_test,(224,224))

In [12]:
x_test_resize.shape 

TensorShape([10000, 224, 224, 1])

In [13]:
class depthwise_separable_conv(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, stride=1):
        super(depthwise_separable_conv, self).__init__()
        self.filters=filters
        self.kernel_size=kernel_size
        self.stride=stride
        
        self.dw=tf.keras.layers.DepthwiseConv2D(kernel_size=self.kernel_size, strides=self.stride, padding='same', use_bias=False)
        self.pw=tf.keras.layers.Conv2D(filters=self.filters, kernel_size=(1,1), strides=1,padding='same',use_bias=False)
        
        self.bn1=tf.keras.layers.BatchNormalization()
        self.bn2=tf.keras.layers.BatchNormalization()
        
        self.act1=tf.keras.layers.Activation('relu')
        self.act2=tf.keras.layers.Activation('relu')
        
    def build(self, input_shape):
        self.batch, self.height, self.width, self.channel = input_shape
    
    def call(self, inputs):
        x=self.dw(inputs)
        x=self.bn1(x)
        x=self.act1(x)
        
        x=self.pw(x)
        x=self.bn2(x)
        x=self.act2(x)
        
        return x

In [14]:
inputs=tf.keras.layers.Input((224,224,1))

x=tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), strides=2, padding='same', use_bias=False)(inputs)
x=depthwise_separable_conv(64,(3,3),1)(x)
x=depthwise_separable_conv(128,(3,3),2)(x)

x=depthwise_separable_conv(128,(3,3),1)(x)
x=depthwise_separable_conv(256,(3,3),2)(x)

x=depthwise_separable_conv(256,(3,3),1)(x)
x=depthwise_separable_conv(512,(3,3),2)(x)

for _ in range(5):
    x=depthwise_separable_conv(512,(3,3),1)(x)

x=depthwise_separable_conv(1024,(3,3),2)(x)

x=depthwise_separable_conv(1024,(3,3),1)(x)

x=tf.keras.layers.GlobalAveragePooling2D()(x)

outputs=tf.keras.layers.Dense(10, activation='softmax')(x)

model=tf.keras.models.Model(inputs=inputs, outputs=outputs)

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 112, 112, 32)      288       
                                                                 
 depthwise_separable_conv (d  (None, 112, 112, 64)     2720      
 epthwise_separable_conv)                                        
                                                                 
 depthwise_separable_conv_1   (None, 56, 56, 128)      9536      
 (depthwise_separable_conv)                                      
                                                                 
 depthwise_separable_conv_2   (None, 56, 56, 128)      18560     
 (depthwise_separable_conv)                                      
                                                             

In [16]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss=tf.keras.losses.SparseCategoricalCrossentropy(),
             metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [17]:
result=model.fit(x_train_resize,
                y_train,
                epochs=10,
                batch_size=16)

Epoch 1/10


2025-07-14 10:00:55.231880: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


   1/3750 [..............................] - ETA: 3:22:35 - loss: 2.4377 - sparse_categorical_accuracy: 0.0625

2025-07-14 10:00:56.295997: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3750/3750 [==============================] - 86s 22ms/step - loss: 0.1367 - sparse_categorical_accuracy: 0.9572
Epoch 2/10
1318/3750 [=========>....................] - ETA: 54s - loss: 0.0668 - sparse_categorical_accuracy: 0.9802

KeyboardInterrupt: 

In [18]:
model.evaluate(x_test_resize, y_test)

313/313 [==============================] - 3s 10ms/step - loss: 0.1639 - sparse_categorical_accuracy: 0.9539


[0.16386274993419647, 0.9538999795913696]